In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import zipfile
import os
import cv2
import numpy as np
import json
import shutil

In [18]:
zip_file_path = '/content/drive/MyDrive/R&E/AI Hub dataset/019.위성영상객체판독/2.Validation/원천데이터_231107_add/validate_roads_images.zip'
extracted_path = '/content/drive/MyDrive/R&E/2. Validation/extracted_images'
png_path='/content/drive/MyDrive/R&E/2. Validation/png_images'
zip_file_path_json = '/content/drive/MyDrive/R&E/AI Hub dataset/019.위성영상객체판독/2.Validation/라벨링데이터_231107_add/validate_roads_labels.zip'
json_path = '/content/drive/MyDrive/R&E/2. Validation/extracted_json'
input_folder = '/content/drive/MyDrive/R&E/2. Validation/png_images'
output_folder = '/content/drive/MyDrive/R&E/2. Validation/split_images'
extracted_json_path = '/content/drive/MyDrive/R&E/2. Validation/extracted_json'
mask_output_folder = "/content/drive/MyDrive/R&E/2. Validation/mask_images"
png_images_folder = '/content/drive/MyDrive/R&E/2. Validation/png_images'

In [16]:
#기존 폴더 삭제
folder_to_delete=['/content/drive/MyDrive/R&E/resized_images','/content/drive/MyDrive/R&E/resized_mask_images']

for folder in folder_to_delete:
    if os.path.exists(folder):
        try:
            shutil.rmtree(folder)
            print(f"Successfully deleted folder: {folder}")
        except OSError as e:
            print(f"Error deleting folder {folder}: {e}")
    else:
        print(f"Folder not found: {folder}")

print("Finished deleting specified folder.")

Successfully deleted folder: /content/drive/MyDrive/R&E/resized_images
Folder not found: /content/drive/MyDrive/R&E/resized_mask_images
Finished deleting specified folder.


In [9]:
#데이터셋 내 이미지 압축 해제
os.makedirs(extracted_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

print(f"Compressed data extracted to: {extracted_path}")

Compressed data extracted to: /content/drive/MyDrive/R&E/2. Validation/extracted_images


In [10]:
#json 파일(마스크 데이터) 압축 해제

os.makedirs(json_path, exist_ok=True)

try:
    with zipfile.ZipFile(zip_file_path_json, 'r') as zip_ref:
        zip_ref.extractall(json_path)
    print(f"Compressed data extracted to: {json_path}")
except FileNotFoundError:
    print(f"Error: The file {zip_file_path_json} was not found.")
except zipfile.BadZipFile:
    print(f"Error: The file {zip_file_path_json} is not a valid zip file.")
except Exception as e:
    print(f"An error occurred: {e}")

Compressed data extracted to: /content/drive/MyDrive/R&E/2. Validation/extracted_json


In [11]:
#PNG 이미지만 모아서 따로 저장
os.makedirs(png_path, exist_ok=True)

processed_count = 0
for file_name in sorted(os.listdir(extracted_path)):
    file_path = os.path.join(extracted_path, file_name)
    if os.path.isfile(file_path) and file_name.lower().endswith('.png'):
        processed_count += 1
        destination_path = os.path.join(png_path, file_name)

        try:
            shutil.copy(file_path, destination_path)
            print(f"Processed {processed_count}: Saved {file_name} to {png_path}")
        except Exception as e:
            print(f"Error processing {processed_count}: Error saving {file_name}: {e}")

print("Finished copying PNG images.")

Processed 1: Saved BLD00002_PS3_K3A_NIA0276.png to /content/drive/MyDrive/R&E/2. Validation/png_images
Processed 2: Saved BLD00010_PS3_K3A_NIA0276.png to /content/drive/MyDrive/R&E/2. Validation/png_images
Processed 3: Saved BLD00029_PS3_K3A_NIA0276.png to /content/drive/MyDrive/R&E/2. Validation/png_images
Processed 4: Saved BLD00047_PS3_K3A_NIA0276.png to /content/drive/MyDrive/R&E/2. Validation/png_images
Processed 5: Saved BLD00072_PS3_K3A_NIA0276.png to /content/drive/MyDrive/R&E/2. Validation/png_images
Processed 6: Saved BLD00086_PS3_K3A_NIA0276.png to /content/drive/MyDrive/R&E/2. Validation/png_images
Processed 7: Saved BLD00092_PS3_K3A_NIA0276.png to /content/drive/MyDrive/R&E/2. Validation/png_images
Processed 8: Saved BLD00097_PS3_K3A_NIA0276.png to /content/drive/MyDrive/R&E/2. Validation/png_images
Processed 9: Saved BLD00108_PS3_K3A_NIA0276.png to /content/drive/MyDrive/R&E/2. Validation/png_images
Processed 10: Saved BLD00119_PS3_K3A_NIA0276.png to /content/drive/MyDriv

In [19]:
# 도로 이미지 512x512 resizing
output_folder = '/content/drive/MyDrive/R&E/2. Validation/resized_images'
os.makedirs(output_folder, exist_ok=True)

image_size = 1024
tile_size = 512
tiles_per_side = image_size // tile_size

for file_name in os.listdir(input_folder):
    if file_name.lower().endswith('.png'):
        file_path = os.path.join(input_folder, file_name)
        try:
            img = cv2.imread(file_path)
            if img is None:
                print(f"Error loading image: {file_name}")
                continue

            resized_img = cv2.resize(img, (tile_size, tile_size), interpolation=cv2.INTER_AREA)

            base_name = os.path.splitext(file_name)[0]
            output_path = os.path.join(output_folder, f"{base_name}.png")

            cv2.imwrite(output_path, resized_img)
            print(f"Resized and saved image: {base_name}.png")

        except Exception as e:
            print(f"Error processing image {file_name}: {e}")


print("Finished resizing and saving images.")

Resized and saved image: BLD00002_PS3_K3A_NIA0276.png
Resized and saved image: BLD00010_PS3_K3A_NIA0276.png
Resized and saved image: BLD00029_PS3_K3A_NIA0276.png
Resized and saved image: BLD00047_PS3_K3A_NIA0276.png
Resized and saved image: BLD00072_PS3_K3A_NIA0276.png
Resized and saved image: BLD00086_PS3_K3A_NIA0276.png
Resized and saved image: BLD00092_PS3_K3A_NIA0276.png
Resized and saved image: BLD00097_PS3_K3A_NIA0276.png
Resized and saved image: BLD00108_PS3_K3A_NIA0276.png
Resized and saved image: BLD00119_PS3_K3A_NIA0276.png
Resized and saved image: BLD00120_PS3_K3A_NIA0276.png
Resized and saved image: BLD00153_PS3_K3A_NIA0276.png
Resized and saved image: BLD00169_PS3_K3A_NIA0276.png
Resized and saved image: BLD00171_PS3_K3A_NIA0276.png
Resized and saved image: BLD00173_PS3_K3A_NIA0276.png
Resized and saved image: BLD00181_PS3_K3A_NIA0276.png
Resized and saved image: BLD00198_PS3_K3A_NIA0276.png
Resized and saved image: BLD00199_PS3_K3A_NIA0276.png
Resized and saved image: BLD

In [20]:
#마스크 이미지 생성
os.makedirs(mask_output_folder, exist_ok=True)

for json_file_name in os.listdir(extracted_json_path):
    if json_file_name.lower().endswith('.json'):
        json_file_path = os.path.join(extracted_json_path, json_file_name)
        image_file_name = json_file_name.replace('.json', '.png')
        image_file_path = os.path.join(png_images_folder, image_file_name)

        try:
            with open(json_file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            if "features" not in data or not data["features"]:
                print(f"No 'features' key or empty features in {json_file_name}.")
                continue

            mask = np.zeros((1024, 1024), dtype=np.uint8)
            has_road_data = False

            for feature in data["features"]:
                props = feature.get("properties", {})
                road_coords = props.get("road_imcoords", "EMPTY")
                if road_coords and road_coords != "EMPTY": # Added check for empty string
                    coords_list = [coord.strip() for coord in road_coords.replace(',', ' ').split() if coord.strip()]
                    if coords_list:
                        try:
                            if len(coords_list) % 2 == 0:
                                pts = np.array([list(map(float, coords_list[i:i+2])) for i in range(0, len(coords_list), 2)])
                                pts = pts.reshape(-1, 2).astype(np.int32)
                                if len(pts) >= 3:
                                    cv2.fillPoly(mask, [pts], color=255)
                                    has_road_data = True
                                else:
                                    print(f"Skipping polygon with less than 3 points in {json_file_name}: {road_coords}")
                            else:
                                print(f"Skipping invalid coordinate list in {json_file_name}: {road_coords} - Uneven number of coordinates")
                        except ValueError as ve:
                            print(f"Skipping invalid coordinates in {json_file_name}: {road_coords} - {ve}")
                        except Exception as poly_e:
                            print(f"Error processing polygon in {json_file_name}: {poly_e}")
                    else:
                        print(f"No valid coordinates found in road_imcoords for {json_file_name}")

            if not has_road_data:
                print(f"No valid road data found in {json_file_name}. Mask file will not be created.")
                continue

            mask_file_name = json_file_name.replace('.json', '_mask.png')
            mask_output_path = os.path.join(mask_output_folder, mask_file_name)
            cv2.imwrite(mask_output_path, mask)
            print(f"Road mask saved as {mask_file_name}")

        except FileNotFoundError:
            print(f"Error: JSON file not found: {json_file_name}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from file: {json_file_name}")
        except Exception as e:
            print(f"An error occurred while processing {json_file_name}: {e}")

print("Finished processing JSON files and saving masks.")

Road mask saved as BLD00002_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00010_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00029_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00047_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00072_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00086_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00092_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00097_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00108_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00119_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00120_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00153_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00169_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00171_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00173_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00181_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00198_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00199_PS3_K3A_NIA0276_mask.png
Road mask saved as BLD00207_PS3_K3A_NIA0276_ma

In [21]:
# 마스크 이미지 512x512 resizing
input_folder = '/content/drive/MyDrive/R&E/2. Validation/mask_images'
output_folder = '/content/drive/MyDrive/R&E/2. Validation/resized_mask_images'
os.makedirs(output_folder, exist_ok=True)

image_size = 1024
tile_size = 512
tiles_per_side = image_size // tile_size

for file_name in os.listdir(input_folder):
    if file_name.lower().endswith('.png'):
        file_path = os.path.join(input_folder, file_name)
        try:
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Error loading mask image: {file_name}")
                continue


            resized_img = cv2.resize(img, (tile_size, tile_size), interpolation=cv2.INTER_AREA)

            base_name = os.path.splitext(file_name)[0]
            output_path = os.path.join(output_folder, f"{base_name}.png")

            cv2.imwrite(output_path, resized_img)
            print(f"Resized and saved mask image: {base_name}.png")

        except Exception as e:
            print(f"Error processing mask image {file_name}: {e}")


print("Finished resizing and saving mask images.")

Resized and saved mask image: BLD00002_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00010_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00029_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00047_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00072_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00086_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00092_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00097_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00108_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00119_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00120_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00153_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00169_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00171_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00173_PS3_K3A_NIA0276_mask.png
Resized and saved mask image: BLD00181_P